In [1]:
%matplotlib inline
import numpy as np
import os

In [2]:
import azureml
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.35.0


In [3]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: mlopsjjk
Azure region: westeurope
Subscription id: 932c3e14-d0cf-4e41-9998-bdebb9bfa1cf
Resource group: networkwatcherrg


In [4]:
from azureml.core import Experiment

script_folder = './scr'
os.makedirs(script_folder, exist_ok=True)

exp = Experiment(workspace=ws, name='myExp')

In [5]:
with open(os.path.join(script_folder, 'myModel.py'), 'r') as f:
    print(f.read())

import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

DATA_PATH="https://raw.githubusercontent.com/urtbest86/MLOps/master/result_train_dataset2.csv"
df = pd.read_csv(DATA_PATH)

DATA_PATH="https://raw.githubusercontent.com/urtbest86/MLOps/master/result_test_dataset2.csv"
test = pd.read_csv(DATA_PATH)

train=df.sample(frac=0.8)
val=df.sample(frac=0.2)

mean = train.mean(axis=0)
train -= mean
std = train.std(axis=0)
train /= std

test -= mean
test /= std

val-=mean
val/=std

train_data_set = train.values
x_train = train_data_set[:, 2:-1].astype(float)
y_train = train_data_set[:, -1].astype(float)

test_data_set = test.values
x_test = test_data_set[:, 2:-1].astype(float)
y_test = test_data_set[:, -1].astype(float)

val_data_set = val.values
x_val = val_data_set[:, 2:-1].astype(float)
y_val = val_data_set[:, -1].astype(float)


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [15]:
%%writefile conda_dependencies.yml

channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - h5py<=2.10.0
  - azureml-defaults
  - tensorflow-gpu==2.0.0
  - keras<=2.3.1
  - matplotlib
  - pandas

Overwriting conda_dependencies.yml


In [16]:
from azureml.core import Environment

keras_env = Environment.from_conda_specification(name = 'keras-2.3.1', file_path = './conda_dependencies.yml')

# Specify a GPU base image
keras_env.docker.enabled = True
keras_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.0-cudnn7-ubuntu18.04'

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [17]:
from azureml.core import ScriptRunConfig

args = ['--data-folder', 
        '--batch-size', 50,
        '--first-layer-neurons', 300,
        '--second-layer-neurons', 100,
        '--learning-rate', 0.001]

src = ScriptRunConfig(source_directory=script_folder,
                      script='myModel.py',
                      arguments=args,
                      environment=keras_env)

In [18]:
run = exp.submit(src)

In [19]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [20]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
myExp,myExp_1634718324_8fadeca6,azureml.scriptrun,Preparing,Link to Azure Machine Learning studio,Link to Documentation


In [21]:
run.wait_for_completion(show_output=True)

RunId: myExp_1634718324_8fadeca6
Web View: https://ml.azure.com/runs/myExp_1634718324_8fadeca6?wsid=/subscriptions/932c3e14-d0cf-4e41-9998-bdebb9bfa1cf/resourcegroups/networkwatcherrg/workspaces/mlopsjjk&tid=19411b3a-b7ca-48d9-a1e2-31694d0ccb65

Streaming azureml-logs/60_control_log.txt

[2021-10-20T08:25:25.652253] Using urllib.request Python 3.0 or later
Streaming log file azureml-logs/60_control_log.txt
Starting the daemon thread to refresh tokens in background for process with pid = 26875
Running: ['/bin/bash', '/tmp/azureml_runs/myExp_1634718324_8fadeca6/azureml-environment-setup/docker_env_checker.sh']

Materialized image not found on target: azureml/azureml_bc47a7211652b5364ba9afa695921e24


[2021-10-20T08:25:26.255089] Logging experiment preparation status in history service.
Running: ['/bin/bash', '/tmp/azureml_runs/myExp_1634718324_8fadeca6/azureml-environment-setup/docker_env_builder.sh']
Running: ['nvidia-docker', 'build', '-f', 'azureml-environment-setup/Dockerfile', '-t',

{'runId': 'myExp_1634718324_8fadeca6',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-10-20T08:28:28.020442Z',
 'endTimeUtc': '2021-10-20T08:31:34.015667Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '4eb4d92a-9a69-4cd5-9ea7-6aa4bccdf2c9'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'myModel.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--data-folder',
   '--batch-size',
   '50',
   '--first-layer-neurons',
   '300',
   '--second-layer-neurons',
   '100',
   '--learning-rate',
   '0.001'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environment': {'name':